### installing GPN


In [11]:
#pip install git+https://github.com/songlab-cal/gpn.git

In [ ]:
### quick start gouid 

In [1]:
import gpn.model
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("songlab/gpn-brassicales")
# or
#model = AutoModelForMaskedLM.from_pretrained("songlab/gpn-msa-sapiens")

/home/johanulstrup/miniconda3/envs/GPN/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
#pip install quarto



In [ ]:
## just a test

In [9]:
#conda install seaborn


In [4]:
### just for fun lets load some data
import sys
import numpy as np
import pandas as pd
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

sns.set_context(context='paper', font_scale=0.8, rc=None)
# sns.set_context(context='notebook', font_scale=1, rc=None)

sns.set_style("ticks", {"axes.spines.top": False, 
                        "axes.spines.right": False,
                       "legend.edgecolor": 'white'})
import random
random_seed = 5

sys.path.append('..')
from global_params import load_params

In [6]:
params = load_params('../global_params.yml')
params

{'sample_size': 24}

In [7]:
params.sample_size

24

In [8]:
subjects = pd.read_csv('../data/data_table.csv')
assert subjects.index.size == params.sample_size
subjects

,name,age,sex,position,nationality
0,Julie,27,F,PhDstudent,DK
1,Thomas,33,M,Postdoc,GB
2,Emilie,23,F,PhDstudent,CH
3,Sofie,31,F,Postdoc,DK
4,Sara,29,F,Postdoc,US
5,Cecilie,34,F,Postdoc,DK
6,Anders,32,M,PhDstudent,UK
7,Emma,42,F,Professor,DK
8,Caroline,31,F,PhDstudent,DK
9,Laura,30,F,Postdoc,DK
